In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
# from catboost import CatBoostRegressor
import re

from sklearn.base import clone
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
url = "https://drive.google.com/u/0/uc?id=1RhXDaw02wlVtvUggA9C81i78vDTM8u14&export=download"
train = pd.read_csv(url) # мой подготовленный датасет для обучения модели
# test = pd.read_csv('test.csv')
# sample_submission = pd.read_csv('sample_submission.csv')

In [4]:
#Смотрим собранные Юрием данные
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  description              37426 non-null  object 
 11  mileage                  37426 non-null  int64  
 12  Привод                   37356 non-null  object 
 13  Руль                     37232 non-null  object 
 14  Состояние             

In [5]:
#Имеем 31 колонку и 37426 строчек
#Посмотрим как данные выглядят и может сразу что то уберем
pd.set_option('display.max_columns', 40)
display(train.head(20))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Аудиосистема,Фары,Климат-контроль,Салон,Диски,autoCatalogUrl,itemPrice,marketPrice,status,closingReason,typeOfTrade,canonicalUrl,avitoId
0,внедорожник,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,Прoдaю лeгендаpный aвтомобиль по ценe китайcко...,180000,полный,Левый,Не битый,3,передние и задние,гидро-,NaN,NaN,Кондиционер,велюр,NaN,/autocatalog/land_rover/discovery/i-19891998_4...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/land_ro...,263245368
1,внедорожник,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,"В нормальном состоянии. Салон кожаный, пяти м...",330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/hyundai/galloper/i-19912003/vnedo...,170000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/hyundai...,633133188
2,седан,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,"Отличный автомобиль, всё есть, надо небольшое ...",124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,8+ колонок,NaN,NaN,NaN,"18""",/autocatalog/mercedes-benz/s-klass/w221-200520...,700000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,624926344
3,хэтчбек,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,"В отличном состоянии, комплетация Elegance c д...",76500,передний,Левый,Не битый,1,передние и задние,электро-,NaN,NaN,климат-контроль многозонный,NaN,"16""",/autocatalog/skoda/octavia/iii-20132017_3291/k...,675000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/skoda_o...,624923120
4,микроавтобус,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,"B наличии, бeз пpобeга Меrсеdеs Sprintеr VIP о...",90,задний,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,кожа,NaN,/autocatalog/mercedes-benz/sprinter_2370/w906-...,10900000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,622789649
5,минивэн,Opel,Серый,Бензин,Zafira,5.0,2012,робот,1.8,140 л.с.,"В хорошем состоянии, находится в сочи",111300,передний,Левый,Не битый,2,передние и задние,электро-,4 колонки,NaN,климат-контроль однозонный,NaN,"16""",/autocatalog/opel/zafira/b-restayling-20082014...,385000,444200.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/opel_za...,633420571
6,седан,Mercedes-Benz,Чёрный,Дизель,E-класс,4.0,2004,автоматическая,2.1,150,....ПРИГНАНА ИЗ ГЕРМАНИИ 2008Г...ЖЕЛЕЗО И КРАС...,380000,задний,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/e-klass/w211s211-20...,340000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,634877947
7,седан,Oldsmobile,Красный,Бензин,Eighty-Eight,4.0,1992,автоматическая,3.8,172 л.с.,Машина не нa ходу. Kоробка в идeалe! двиготeль...,150000,передний,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/oldsmobile/eighty-eight/x-1992199...,80000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/oldsmob...,621276255
8,внедорожник,Mercedes-Benz,Золотой,Бензин,M-класс AMG,5.0,2002,автоматическая,5.4,347,Пpодaм Mеpсeдес Мл 320 АMG в кузовe (w163) тех...,160000,полный,Левый,Не битый,1,передние и задние,гидро-,NaN,NaN,NaN,NaN,NaN,/autocatalog/mercedes-benz/m-klass_amg/w163-re...,500000,NaN,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/mercede...,627205114
9,внедорожник,Volkswagen,Серый,Дизель,Touareg,5.0,2008,автоматическая,3.0,240 л.с.,Сpoчнo!!! Maшина в отличном техничеcком cоcтоя...,222000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/autocatalog/volkswagen/touareg/i-restayling-2...,715000,796600.0,NaN,NaN,Продаю личный автомобиль,https://www.avito.ru/moskva/avtomobili/volkswa...,629324840


In [6]:
#На первый взгляд видим следующее:
#1. enginePower- нужно будет убрать "л.с."
#2. description- предлагаю полностью убрать это столбец
#3. autoCatalogUrl, status, closingReason, canonicalUrl, avitoId, Аудиосистема, Фары - можно спокойно удалять. Такие значения
# как аудиосистема или фары, практически не вляют на ценник, да и информации по ним мало.
#4. С остальными полями будем разбираться дальше.
train.drop(['autoCatalogUrl', 'status', 'closingReason','canonicalUrl','avitoId','Аудиосистема','Фары', 'description'], inplace = True, axis = 1)

In [7]:
#Проверям, смотрим. Осталось 23 колонки. Теперь пройдемся по каждой
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37426 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37426 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37426 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37128 non-null  object 
 9   enginePower              37410 non-null  object 
 10  mileage                  37426 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37232 non-null  object 
 13  Состояние                37426 non-null  object 
 14  Владельцы             

In [8]:
#Посмотрим какие уникальные значения в столбце bodyType
train.bodyType.value_counts()

седан           14412
внедорожник      9808
хэтчбек          6198
минивэн          1926
универсал        1693
фургон           1591
купе              834
пикап             354
микроавтобус      347
кабриолет         193
Name: bodyType, dtype: int64

In [9]:
#все хорошо есть, но есть 70 пропусков, предлагаю их убрать. Еще одно НО, посмотрим на них полностью
temp = train[train.bodyType.isnull()]
display(temp.head(30))

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
1068,NaN,NaN,Фиолетовый,NaN,NaN,NaN,2012,NaN,NaN,117 л.с.,43750,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,650000,NaN,Продаю личный автомобиль
1234,NaN,NaN,Серебряный,NaN,NaN,NaN,2007,NaN,NaN,248 л.с.,195000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,399000,497400.0,Продаю личный автомобиль
1380,NaN,NaN,Красный,NaN,NaN,NaN,1994,NaN,NaN,90 л.с.,198500,NaN,Левый,Не битый,4+,NaN,NaN,NaN,NaN,NaN,87000,94700.0,Автомобиль приобретён на продажу
1448,NaN,NaN,Белый,NaN,NaN,NaN,2011,NaN,NaN,163 л.с.,221000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,810000,957700.0,Продаю личный автомобиль
1480,NaN,NaN,Чёрный,NaN,NaN,NaN,2012,NaN,NaN,170 л.с.,143000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1000000,974300.0,Продаю личный автомобиль
1484,NaN,NaN,Чёрный,NaN,NaN,NaN,2013,NaN,NaN,249 л.с.,77000,NaN,Левый,Не битый,1,NaN,NaN,NaN,NaN,NaN,1100000,1109800.0,Продаю личный автомобиль
1818,NaN,NaN,Чёрный,NaN,NaN,NaN,2014,NaN,NaN,245 л.с.,77000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,1630000,NaN,Продаю личный автомобиль
2141,NaN,NaN,Серый,NaN,NaN,NaN,2001,NaN,NaN,65 л.с.,202000,NaN,Правый,Не битый,2,NaN,NaN,NaN,NaN,NaN,80000,NaN,Продаю личный автомобиль
2193,NaN,NaN,Чёрный,NaN,NaN,NaN,2016,NaN,NaN,135 л.с.,35000,NaN,Левый,Не битый,2,NaN,NaN,NaN,NaN,NaN,599000,572000.0,Продаю личный автомобиль
2197,NaN,NaN,Синий,NaN,NaN,NaN,2006,NaN,NaN,125 л.с.,156000,NaN,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,250000,249200.0,Автомобиль приобретён на продажу


In [10]:
#Видим, что информации по ним 0, удаляем
train.dropna(subset = ['bodyType'],inplace = True)

In [11]:
#число полей сократилось на 70
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37356 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  object 
 1   brand                    37356 non-null  object 
 2   color                    37356 non-null  object 
 3   fuelType                 37356 non-null  object 
 4   name                     37356 non-null  object 
 5   numberOfDoors            37355 non-null  float64
 6   productionDate           37356 non-null  int64  
 7   vehicleTransmission      37356 non-null  object 
 8   engineDisplacement       37122 non-null  object 
 9   enginePower              37340 non-null  object 
 10  mileage                  37356 non-null  int64  
 11  Привод                   37356 non-null  object 
 12  Руль                     37162 non-null  object 
 13  Состояние                37356 non-null  object 
 14  Владельцы             

In [13]:
#Создадим словарь типов машин, чтобы использовать метод map
dict_bodyType= {'седан': 1,
'внедорожник': 2,
'хэтчбек': 3,
'минивэн': 4,
'универсал': 5,
'фургон': 6,
'купе': 7,
'пикап': 8,
'микроавтобус' : 9,
'кабриолет' : 10}

In [14]:
#Заменяем
train['bodyType'] = train['bodyType'].map(dict_bodyType)

In [15]:
#Смотрим
train.head()

,bodyType,brand,color,fuelType,name,numberOfDoors,productionDate,vehicleTransmission,engineDisplacement,enginePower,mileage,Привод,Руль,Состояние,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,typeOfTrade
0,2,Land Rover,Чёрный,Бензин,Discovery,5.0,1995,механическая,3.9,182 л.с.,180000,полный,Левый,Не битый,3,передние и задние,гидро-,Кондиционер,велюр,NaN,170000,NaN,Продаю личный автомобиль
1,2,Hyundai,Зелёный,Бензин,Galloper,5.0,1997,автоматическая,3.0,141 л.с.,330000,полный,Левый,Не битый,3,NaN,NaN,NaN,NaN,NaN,170000,NaN,Продаю личный автомобиль
2,1,Mercedes-Benz,Чёрный,Бензин,S-класс,4.0,2008,автоматическая,5.5,388 л.с.,124000,полный,Левый,Не битый,2,передние и задние,электрогидро-,NaN,NaN,"18""",700000,NaN,Продаю личный автомобиль
3,3,Skoda,Белый,Бензин,Octavia,5.0,2013,робот,1.4,140 л.с.,76500,передний,Левый,Не битый,1,передние и задние,электро-,климат-контроль многозонный,NaN,"16""",675000,NaN,Продаю личный автомобиль
4,9,Mercedes-Benz,Чёрный,Бензин,Sprinter,4.0,2019,автоматическая,3.5,258 л.с.,90,задний,Левый,Не битый,1,NaN,NaN,NaN,кожа,NaN,10900000,NaN,Продаю личный автомобиль


# Смотрим "brand"

In [16]:
#Получаем, что у нас 123 различных брендов. Пойдем по пути, что в ТЕСТОВОМ у нас одни BMW
train.brand.value_counts()

Mercedes-Benz      3002
BMW                2519
Volkswagen         2375
Hyundai            2335
ВАЗ (LADA)         2302
                   ... 
Derways               1
Tata                  1
Doninvest             1
Renault Samsung       1
Комбат                1
Name: brand, Length: 123, dtype: int64

In [15]:
#Оставим одни значения BMW
#train_BMW = train[train['brand'].isin(['BMW'])]

# Обработка категориальных признаков "скопом"

# color
# vehicleTransmission
# Привод
# Состояние
# Руль
# Name
# fuelType
# typeOfTrade

In [17]:
#В столбце "Руль" есть пропуски, убираем.
print(train.Руль.value_counts())
print(train.Руль.isna().sum())
fs = train[train.Руль.isnull()]

Левый     36017
Правый     1145
Name: Руль, dtype: int64
194


In [18]:
#Посмотрим в каких брендах у нас пропуски и мы точно знаем, что они имеют "левый" руль
fs.brand.value_counts()

Hyundai          16
Ford             14
Volkswagen       13
KIA              13
ВАЗ (LADA)       12
Mercedes-Benz    12
Nissan           10
Renault          10
Peugeot           9
Land Rover        8
Chevrolet         8
Skoda             8
Toyota            7
BMW               7
Mitsubishi        6
Mazda             6
Volvo             6
Opel              5
Audi              5
Lexus             4
УАЗ               2
Chery             2
Datsun            2
SsangYong         2
Infiniti          2
Citroen           2
Subaru            1
Daewoo            1
Jeep              1
Name: brand, dtype: int64

In [19]:
train.loc[(train['brand'] == "BMW") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Renault") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Volkswagen") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Peugeot") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Chevrolet") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Audi") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Opel") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Lexus") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Skoda") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train['brand'] == "Mercedes-Benz") & (train.Руль.isnull()) , 'Руль'] = "Левый"
train.loc[(train.Руль.isnull()) , 'Руль'] = "Правый"

In [20]:
#Нашел статью на хабре, пробуем https://habr.com/ru/post/456294/  , а то в прошлый раз все словарями делал. 
LE = LabelEncoder()
LE.fit(train['color'])
train['Color_new'] = LE.transform(train['color'])
train.drop(['color'],axis = 1, inplace = True)
LE.fit(train['vehicleTransmission'])
train['vehicleTransmission_new'] = LE.transform(train['vehicleTransmission'])
train.drop(['vehicleTransmission'],axis = 1, inplace = True)
LE.fit(train['Привод'])
train['Привод_new'] = LE.transform(train['Привод'])
train.drop(['Привод'],axis = 1, inplace = True)
LE = LabelEncoder()
#LE.fit(train['Руль']) Выдает ошибку, может кто то поймет как с ней бороться
#train['Руль_new'] = LE.transform(train['Руль'])
#train.drop(['Руль'],axis = 1, inplace = True)
LE.fit(train['Состояние'])
train['Состояние_new'] = LE.transform(train['Состояние'])
train.drop(['Состояние'],axis = 1, inplace = True)
LE.fit(train['name'])
train['name_new'] = LE.transform(train['name'])
train.drop(['name'],axis = 1, inplace = True)
LE.fit(train['fuelType'])
train['fuelType_new'] = LE.transform(train['fuelType'])
train.drop(['fuelType'],axis = 1, inplace = True)
LE.fit(train['typeOfTrade'])
train['typeOfTrade_new'] = LE.transform(train['typeOfTrade'])
train.drop(['typeOfTrade'],axis = 1, inplace = True)

In [21]:
dict_rul={"Левый": 1, "Правый": 2}
train['Руль'] = train['Руль'].map(dict_rul)

# Столбец "enginePower"

In [25]:
#Чистим от л.с.
train['enginePower'] = train['enginePower'].apply(lambda x: float(str(x).split()[0]))

In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37356 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37356 non-null  int64  
 1   brand                    37356 non-null  object 
 2   numberOfDoors            37355 non-null  float64
 3   productionDate           37356 non-null  int64  
 4   engineDisplacement       37122 non-null  object 
 5   enginePower              37340 non-null  float64
 6   mileage                  37356 non-null  int64  
 7   Руль                     37356 non-null  int64  
 8   Владельцы                37356 non-null  object 
 9   Электростеклоподъемники  5839 non-null   object 
 10  Усилитель руля           5808 non-null   object 
 11  Климат-контроль          3310 non-null   object 
 12  Салон                    2219 non-null   object 
 13  Диски                    4922 non-null   object 
 14  itemPrice             

In [24]:
train.head()

,bodyType,brand,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new
0,2,Land Rover,5.0,1995,3.9,182.0,180000,1,3,передние и задние,гидро-,Кондиционер,велюр,NaN,170000,NaN,15,2,2,1,445,0,1
1,2,Hyundai,5.0,1997,3.0,141.0,330000,1,3,NaN,NaN,NaN,NaN,NaN,170000,NaN,4,0,2,1,586,0,1
2,1,Mercedes-Benz,4.0,2008,5.5,388.0,124000,1,2,передние и задние,электрогидро-,NaN,NaN,"18""",700000,NaN,15,0,2,1,983,0,1
3,3,Skoda,5.0,2013,1.4,140.0,76500,1,1,передние и задние,электро-,климат-контроль многозонный,NaN,"16""",675000,NaN,1,3,1,1,836,0,1
4,9,Mercedes-Benz,4.0,2019,3.5,258.0,90,1,1,NaN,NaN,NaN,кожа,NaN,10900000,NaN,15,0,0,1,1069,0,1


# Столбец "engineDisplacement"

In [23]:
#Видим пропуски, но не вижу пока способа их подставить.
print(train.engineDisplacement.isna().sum())

234


In [24]:
train.dropna(subset = ['engineDisplacement'],inplace = True)

In [25]:
train["engineDisplacement"].unique()

array(['3.9', '3.0', '5.5', '1.4', '3.5', '1.8', '2.1', '3.8', '5.4',
       '2.4', '3.2', '4.5', '2.0', '1.6', '2.9', '2.5', '4.8', '2.2',
       '4.4', '2.7', '2.3', '3.3', '4.0', '6.0+', '5.9', '1.5', '3.6',
       '2.8', '1.7', '5.7', '4.2', '1.3', '0.8', '6.0', '5.8', '0.7',
       '4.1', '1.0', '5.6', '1.2', '4.6', '5.0', '3.7', '4.7', '2.6',
       '5.3', '3.1', '1.9', '1.1', '4.3', '5.2', '0.6', '3.4', '4.9',
       '0.9', '5.1'], dtype=object)

In [26]:
train.loc[(train["engineDisplacement"] == "6.0+") , 'engineDisplacement'] = "7.0"

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37122 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37122 non-null  int64  
 1   brand                    37122 non-null  object 
 2   numberOfDoors            37121 non-null  float64
 3   productionDate           37122 non-null  int64  
 4   engineDisplacement       37122 non-null  object 
 5   enginePower              37122 non-null  float64
 6   mileage                  37122 non-null  int64  
 7   Руль                     37122 non-null  int64  
 8   Владельцы                37122 non-null  object 
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

In [28]:
#Пока не знаю как округлить

In [29]:
#После того, как дропнули тут пустые значения, меняем тип данных у "enginePower"
train.enginePower = train.enginePower.astype(np.int32)

# Столбец "numberOfDoors" переводим в int

In [30]:
train.numberOfDoors.value_counts()

5.0    18323
4.0    15731
2.0     1898
3.0     1169
Name: numberOfDoors, dtype: int64

In [31]:
train[train.numberOfDoors.isna()]
train.dropna(subset = ['numberOfDoors'],inplace = True)

In [32]:
train.numberOfDoors = train.numberOfDoors.astype(np.int32)

In [33]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   brand                    37121 non-null  object 
 2   numberOfDoors            37121 non-null  int32  
 3   productionDate           37121 non-null  int64  
 4   engineDisplacement       37121 non-null  object 
 5   enginePower              37121 non-null  int32  
 6   mileage                  37121 non-null  int64  
 7   Руль                     37121 non-null  int64  
 8   Владельцы                37121 non-null  object 
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

# Столбец "Владельцы"

In [34]:
train.Владельцы.value_counts()
#Видим что есть 4+, изменим на просто 4

2     11146
1     10003
3      9841
4+     6131
Name: Владельцы, dtype: int64

In [35]:
train.loc[(train["Владельцы"] == "4+") , 'Владельцы'] = 4

In [36]:
train.Владельцы = train.Владельцы.astype(np.int32)

In [37]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37121 entries, 0 to 37425
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bodyType                 37121 non-null  int64  
 1   brand                    37121 non-null  object 
 2   numberOfDoors            37121 non-null  int32  
 3   productionDate           37121 non-null  int64  
 4   engineDisplacement       37121 non-null  object 
 5   enginePower              37121 non-null  int32  
 6   mileage                  37121 non-null  int64  
 7   Руль                     37121 non-null  int64  
 8   Владельцы                37121 non-null  int32  
 9   Электростеклоподъемники  5799 non-null   object 
 10  Усилитель руля           5761 non-null   object 
 11  Климат-контроль          3294 non-null   object 
 12  Салон                    2213 non-null   object 
 13  Диски                    4887 non-null   object 
 14  itemPrice             

# Столбцы 'Электростеклоподъемники', "Усилитель руля","Климат-контроль", "Салон", "Диски", "marketPrice"
Какие у вас предложения? оставляем или дропаем?

In [27]:

train.head()

,bodyType,brand,numberOfDoors,productionDate,engineDisplacement,enginePower,mileage,Руль,Владельцы,Электростеклоподъемники,Усилитель руля,Климат-контроль,Салон,Диски,itemPrice,marketPrice,Color_new,vehicleTransmission_new,Привод_new,Состояние_new,name_new,fuelType_new,typeOfTrade_new
0,2,Land Rover,5.0,1995,3.9,182.0,180000,1,3,передние и задние,гидро-,Кондиционер,велюр,NaN,170000,NaN,15,2,2,1,445,0,1
1,2,Hyundai,5.0,1997,3.0,141.0,330000,1,3,NaN,NaN,NaN,NaN,NaN,170000,NaN,4,0,2,1,586,0,1
2,1,Mercedes-Benz,4.0,2008,5.5,388.0,124000,1,2,передние и задние,электрогидро-,NaN,NaN,"18""",700000,NaN,15,0,2,1,983,0,1
3,3,Skoda,5.0,2013,1.4,140.0,76500,1,1,передние и задние,электро-,климат-контроль многозонный,NaN,"16""",675000,NaN,1,3,1,1,836,0,1
4,9,Mercedes-Benz,4.0,2019,3.5,258.0,90,1,1,NaN,NaN,NaN,кожа,NaN,10900000,NaN,15,0,0,1,1069,0,1
